In [ ]:
import os.path
import sys
import time
from datetime import datetime, timedelta
import pandas as pd

from func.tdx_func import *
from func.func_sys import *
from easyquant  import MongoIo
from multiprocessing import Process, Pool, cpu_count, Manager

import baostock as bs
import pandas as pd

minutes_field = "date,time,code,open,high,low,close,volume,amount,adjustflag"
day_field = "date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM"
w_m_field = "date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg"
profile_field = 'code,pubDate,statDate,roeAvg,npMargin,gpMargin,netProfit,epsTTM,MBRevenue,totalShare,liqaShare'
WEEK = 'w'
MONTH = 'm'
QFQ = '2'
HFQ = '1'
BFQ = '3'

databuf_mongo = Manager().dict()

def get_next_date(calcDate):
    cDate = datetime.datetime.strptime(calcDate, '%Y-%m-%d')
    cDate = cDate + datetime.timedelta(1)
    if cDate.weekday() < 5:
        return datetime.datetime.strftime(cDate,'%Y-%m-%d')
    else: # if calcDate.weekday() == 5:
        cDate = cDate + datetime.timedelta(2)
        return datetime.datetime.strftime(cDate,'%Y-%m-%d')

def fetch_k_day(code="sh.600606", p_begin_day: str = '2023-01-01', p_end_day: str = None):
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg,
    # 换手率\涨跌幅（百分比）\滚动市盈率\市净率\滚动市销率\滚动市现率\
    # 日周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM

    # frequency：数据类型，默认为d，日k线；d = 日k线、w = 周、m = 月、5 = 5
    # 分钟、15 = 15
    # 分钟、30 = 30
    # 分钟、60 = 60
    # 分钟k线数据，不区分大小写；指数没有分钟线数据；周线每周最后一个交易日才可以获取，月线每月最后一个交易日才可以获取

    # adjustflag：复权类型，默认不复权：3；1：后复权；2：前复权。已支持分钟线、日线、周线、月线前后复权
    if p_end_day is None:
        p_end_day = time.strftime('%Y-%m-%d')
#     print(p_begin_day, p_end_day)
    rs = bs.query_history_k_data_plus(code,
                                      day_field,
                                      start_date=p_begin_day, end_date=p_end_day,
                                      frequency="d", adjustflag=QFQ)
    data_list = []
#     if rs is None:
#         return pd.DataFrame()
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    # 添加方式
    # result.to_csv(p_file, index=False, mode='a+', header=True)
    return result

def update_data(codelist):
    start_t = datetime.datetime.now()
    print("begin-update_data:", start_t)
    mongo_mpd = MongoIo()
    tblName = 'stock_day_qfq'
    pool_size = cpu_count()
    code_dict = codelist2dict(codelist, pool_size)
    j = 0
#     for i in range(4,5): #code_dict.keys():
    for i in code_dict.keys():
#         pool.apply_async(do_update_data_mp, args=(i, code_dict[i]))
        for xcode in code_dict[i]:
            j += 1
            print("do update data for ", xcode, "... ...", j, len(codelist))
            do_update_data_mp(tblName, mongo_mpd, xcode, i)
    end_t = datetime.datetime.now()
    # print("data-total-len:", len(dataR))
    print(end_t, 'update_data spent:{}'.format((end_t - start_t)))

def ins_mongo_data(mongo_mpd, tdata, code):
    if len(tdata) > 0:
        akey = tdata.columns.values
        ins_data = []
        for index, row in tdata.iterrows():
            monv = {}
            for x in akey:
                if x == 'code':
                    monv[x] = row[x][3:]
                elif x == 'date':
                    monv[x] = row[x]
                else:
                    try:
                        monv[x] = float(row[x])
                    except:
                        monv[x] = 0.0
    #                                 print("conv error", code, row)

            monv['_id'] = '%s-%s' % (row['code'], row['date'])
            monv['date_stamp'] = mongo_mpd.dateStr2stamp(row['date'])
    #                     print(monv)
            ins_data.append(monv)
        #     print(row.tolist())
        if len(ins_data) > 0:
            try:
                mongo_mpd.save(tblName, ins_data)
            except:
                print("ins error", code)
    
def do_update_data_mp(tblName, mongo_mpd, xcode, key):
    tblName = 'stock_day_qfq'
    if '0' == xcode[:1] or '3' == xcode[:1]:
        code = "sz.%s" % xcode
#             print("sz.%s" % x)
    else:
        code = "sh.%s" % xcode
    if len(databuf_mongo[key]) == 0:
        codeData = []
    else:
        codeData = databuf_mongo[key].query(" code == '%s'" % xcode)
    ##debug
#     if xcode == '300088':
#         print(key, xcode, len(codeData), len(databuf_mongo[key]))
        
    if len(codeData) > 0:
        today = datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%')
        p_begin_day = codeData.index[-1][0].strftime("%Y-%m-%d")
        p_begin_day = get_next_date(p_begin_day)
        p_begin_year = p_begin_day[:4]
        p_end_day = '%s-12-31' % p_begin_year
#         if p_begin_day > p_end_day:
#             p_end_day = '%s-12-31' % p_begin_year
        if p_begin_day > today:
            ##last data
            return
        year = datetime.datetime.strftime(datetime.datetime.now(),'%Y')
        for ldate in range(p_begin_year,int(year)+1):
            if ldate > p_begin_year:
                p_begin_day = '%s-01-01' % ldate
                p_end_day = '%s-12-31' % ldate
            tdata = fetch_k_day(code, p_begin_day = p_begin_day, p_end_day = p_end_day)
            ins_mongo_data(mongo_mpd, tdata, xcode)
    else:
        year = datetime.datetime.strftime(datetime.datetime.now(),'%Y')
        for ldate in range(1990,int(year)+1):
            p_begin_day = '%s-01-01' % ldate
            p_end_day = '%s-12-31' % ldate
            tdata = fetch_k_day(code, p_begin_day = p_begin_day, p_end_day = p_end_day)
            ins_mongo_data(mongo_mpd, tdata, xcode)

def get_data(codelist):
    start_t = datetime.datetime.now()
    year = datetime.datetime.strftime(datetime.datetime.now(),'%Y')
    print("begin-get_data:", start_t)
    pool_size = cpu_count()
    code_dict = codelist2dict(codelist, pool_size)
    # print("get-data", code_dict)
    pool = Pool(cpu_count())
    for i in code_dict.keys():
        pool.apply_async(do_get_data_mp, args=(i, code_dict[i], '%s-01-01' % year, '%s-12-31' % year))

    pool.close()
    pool.join()

    end_t = datetime.datetime.now()
    # print("data-total-len:", len(dataR))
    print(end_t, 'get_data spent:{}'.format((end_t - start_t)))
    # return data_day
def do_get_data_mp(key, codelist, st_start, st_end):
#     print("do_get_data_mp", key)
    mongo_mp = MongoIo()
#     slip  = 1
    # start_t = datetime.datetime.now()
    # print("begin-get_data do_get_data_mp: key=%s, time=%s" %( key,  start_t))
#     if len(func_nameA) < 1:
#         return
        # 取得当前数据
#     func_buy = func_nameA[0]
#     func_sell = func_nameA[1]
#     databuf_mongo = mongo_mp.get_stock_day(codelist, st_start=st_start, st_end = st_end)
#     print("code-list", codelist[:100])
    try:
        databuf_mongo[key] = mongo_mp.get_stock_day(codelist, st_start=st_start, st_end = st_end, qfq=1)
        print('data-get', key, len(databuf_mongo[key]), st_start, st_end)
    except Error:
        print("do_get_data_mp error", key, Error)
        
    
if __name__ == '__main2__':
    # 登陆系统
    tblName = 'stock_day_qfq'
    lg = bs.login()
    # 显示登陆返回信息
    if lg.error_code != '0':
        print('login respond error_code:' + lg.error_code)
        print('login respond  error_msg:' + lg.error_msg)
        sys.exit()
    
    codelist = getCodeList('all')
    get_data(codelist)
    update_data(codelist)
#     for x in codelist:
#         if '0' == x[:1]:
#             code = "sz.%s" % x
# #             print("sz.%s" % x)
#         else:
#             code = "sz.%s" % x
#         codeData = testData.query(" code == '%s'" % x)
#         if len(codeData) == 0:
#             for ldate in range(1990,2025):
#                 p_begin_day = '%s-01-01' % ldate
#                 p_end_day = '%s-12-31' % ldate
#                 tdata = fetch_k_day(code, p_begin_day = p_begin_day, p_end_day = p_end_day)
#                 if len(tdata) > 0:
#                     akey = tdata.columns.values
#                     ins_data = []
#                     for index, row in tdata.iterrows():
#                         monv = {}
#                         for x in akey:
#                             if x == 'code':
#                                 monv[x] = row[x][3:]
#                             elif x == 'date':
#                                 monv[x] = row[x]
#                             else:
#                                 try:
#                                     monv[x] = float(row[x])
#                                 except:
#                                     monv[x] = 0.0
#     #                                 print("conv error", code, row)

#                         monv['_id'] = '%s-%s' % (row['code'], row['date'])
#                         monv['date_stamp'] = mongo.dateStr2stamp(row['date'])
#     #                     print(monv)
#                         ins_data.append(monv)
#                     #     print(row.tolist())
#                     mongo.save(tblName, ins_data)
#         else:
#             for ldate in range(1990,2025):
#                 tdata = fetch_k_day(code, p_begin_day = ldate)
#                 if len(tdata) > 0:
#                     akey = tdata.columns.values
#                     ins_data = []
#                     for index, row in tdata.iterrows():
#                         monv = {}
#                         for x in akey:
#                             if x == 'code':
#                                 monv[x] = row[x][3:]
#                             elif x == 'date':
#                                 monv[x] = row[x]
#                             else:
#                                 try:
#                                     monv[x] = float(row[x])
#                                 except:
#                                     monv[x] = 0.0
#     #                                 print("conv error", code, row)

#                         monv['_id'] = '%s-%s' % (row['code'], row['date'])
#                         monv['date_stamp'] = mongo.dateStr2stamp('2022-12-12')
#     #                     print(monv)
#                         ins_data.append(monv)
#                     #     print(row.tolist())
#                     mongo.save(tblName, ins_data)
            

#                 print("ok", len(tdata))
#                 print(len(tdata))
#                 break
#     print(codelist[:10])


In [ ]:
tblName = 'stock_day_qfq'
lg = bs.login()
# 显示登陆返回信息
if lg.error_code != '0':
    print('login respond error_code:' + lg.error_code)
    print('login respond  error_msg:' + lg.error_msg)
    sys.exit()

codelist = getCodeList('all')
get_data(codelist)


In [ ]:
xcode = '300765'
key = 0
tblName = 'stock_day_qfq'
if '0' == xcode[:1] or '3' == xcode[:1]:
    code = "sz.%s" % xcode
#             print("sz.%s" % x)
else:
    code = "sh.%s" % xcode
if len(databuf_mongo[key]) == 0:
    codeData = []
else:
    codeData = databuf_mongo[key].query(" code == '%s'" % xcode)


In [ ]:
codeData

In [ ]:
# if len(codeData) > 0:
today = datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d')
p_begin_day = codeData.index[-1][0].strftime("%Y-%m-%d")
p_begin_day = get_next_date(p_begin_day)
p_begin_year = int(p_begin_day[:4])
p_end_day = '%s-12-31' % p_begin_year
#         if p_begin_day > p_end_day:
#             p_end_day = '%s-12-31' % p_begin_year
if p_begin_day > today:
    ##last data
#     return
    pass
year = datetime.datetime.strftime(datetime.datetime.now(),'%Y')
for ldate in range(int(p_begin_year),int(year)+1):
    if ldate > p_begin_year:
        p_begin_day = '%s-01-01' % ldate
        p_end_day = '%s-12-31' % ldate
    tdata = fetch_k_day(code, p_begin_day = p_begin_day, p_end_day = p_end_day)
#     print(tdata)
    for index, row in tdata.iterrows():
        if row['volume'] == '':
            print(' stop buy sell')
            
            continue
#     ins_mongo_data(mongo_mpd, tdata, xcode)
# else:
#     year = datetime.datetime.strftime(datetime.datetime.now(),'%Y')
#     for ldate in range(1990,int(year)+1):
#         p_begin_day = '%s-01-01' % ldate
#         p_end_day = '%s-12-31' % ldate
#         tdata = fetch_k_day(code, p_begin_day = p_begin_day, p_end_day = p_end_day)
#         ins_mongo_data(mongo_mpd, tdata, xcode)


In [ ]:
p_end_day = '%s-12-31' % 'p_begin_year'
p_end_day

In [ ]:
def dateStr2stamp(dateObj):
    dateStr = str(dateObj)[0:10]
    date = time.mktime(time.strptime(dateStr, '%Y-%m-%d'))
    return date


In [ ]:
# dateStr2stamp('2020-12-12')
st_end = '1991-05-03'
st_start='1991-04-03'
code=['000001']

In [ ]:
dateStr2stamp(st_start)

In [ ]:
cursor = db['stock_day_qfq'].find(
    {
        'code': {
            '$in': code
        },
        "date_stamp":
            {
                "$lte": dateStr2stamp(st_end),
                "$gte": dateStr2stamp(st_start)
            }
    },
    {"_id": 0},
    batch_size=10000
)
# res = pd.DataFrame([item for item in cursor])
# try:
#     res = res.assign(
#         volume=res.vol,
#         date=pd.to_datetime(res.date)
#     ).drop_duplicates((['date',
#                         'code'])).query('volume>1').set_index(
#         'date',
#         drop=False
#     )
#     res = res.loc[:,
#           [
#               'code',
#               'open',
#               'high',
#               'low',
#               'close',
#               'volume',
#               'amount',
#               'date'
#           ]]


In [ ]:
for item in cursor:
    print(item)
    break

In [ ]:
res = pd.DataFrame([item for item in cursor])


In [ ]:
res

In [ ]:
today = datetime.strftime(datetime.now(),'%Y')

In [ ]:
today

In [ ]:
codelist = getCodeList('all')

In [ ]:
get_data(codelist)

In [ ]:
lg = bs.login()

In [ ]:
len(codelist)

In [47]:
pool_size = cpu_count()
code_dict = codelist2dict(codelist, pool_size)


In [50]:
for x in code_dict[4]:
    print(x)

000008
000019
000030
000040
000058
000068
000099
000159
000403
000415
000426
000504
000516
000526
000536
000546
000555
000566
000581
000595
000605
000622
000631
000651
000665
000680
000691
000707
000717
000726
000737
000757
000776
000788
000798
000810
000820
000830
000851
000863
000881
000892
000902
000913
000925
000933
000951
000961
000970
000982
000995
001206
001215
001226
001238
001267
001287
001306
001318
001331
001360
001380
002001
002011
002022
002031
002039
002047
002056
002064
002074
002082
002093
002101
002109
002119
002127
002135
002144
002153
002162
002170
002178
002186
002194
002203
002211
002219
002228
002236
002244
002253
002264
002272
002280
002288
002297
002305
002314
002322
002330
002338
002346
002354
002363
002371
002379
002387
002395
002403
002412
002421
002429
002438
002446
002456
002465
002474
002484
002493
002505
002514
002522
002531
002539
002547
002555
002563
002573
002581
002590
002599
002608
002616
002626
002634
002642
002650
002659
002667
002675
002685
002693

In [ ]:
def ins_mongo_data(tdata):
    if len(tdata) > 0:
        akey = tdata.columns.values
        ins_data = []
        for index, row in tdata.iterrows():
            monv = {}
            for x in akey:
                if x == 'code':
                    monv[x] = row[x][3:]
                elif x == 'date':
                    monv[x] = row[x]
                else:
                    try:
                        monv[x] = float(row[x])
                    except:
                        monv[x] = 0.0
    #                                 print("conv error", code, row)

            monv['_id'] = '%s-%s' % (row['code'], row['date'])
            monv['date_stamp'] = mongo_mpd.dateStr2stamp(row['date'])
    #                     print(monv)
            ins_data.append(monv)
        #     print(row.tolist())
        if len(ins_data) > 0:
            mongo_mpd.save(tblName, ins_data)


In [ ]:
code = 'sz.000001'
p_begin_day = '1991-01-01'
p_end_day = '1991-12-31'
tdata = fetch_k_day(code, p_begin_day = p_begin_day, p_end_day = p_end_day)  

In [ ]:
mongo_mpd.get_stock_day(['000001'],qfq=1)

In [ ]:
year='2000'
'%s-12-31' % (int(year) + 1)

In [53]:
year = datetime.datetime.strftime(datetime.datetime.now(),'%Y')

In [54]:
year

'2024'

In [57]:
a = '123'
a != '123'

False